<a href="https://colab.research.google.com/github/Roshelvimukthi/AgroSense/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================
# AgroSense: AI Model Development
# ===============================

# Install required libraries
!pip install tensorflow keras efficientnet scikit-learn pymongo pandas matplotlib seaborn

# ===============================
# 1️⃣ Import Dependencies
# ===============================
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import efficientnet.tfkeras as efn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pymongo import MongoClient
import os

# ===============================
# 2️⃣ Connect to MongoDB Atlas
# ===============================
# Replace with your MongoDB Atlas connection string
MONGO_URI = "mongodb+srv://<username>:<password>@cluster0.mongodb.net/agrosense"
client = MongoClient(MONGO_URI)
db = client["AgroSenseDB"]
collection = db["Predictions"]

print("✅ Connected to MongoDB successfully!")

# ===============================
# 3️⃣ Part A: Plant Disease Detection (CNN)
# ===============================

# Simulated dataset (replace this with actual dataset like PlantVillage)
# Directory structure: dataset/train/<class_name>/image.jpg
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/content/dataset/train"
val_dir = "/content/dataset/validation"

img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Build CNN Model using EfficientNetB0
cnn_model = models.Sequential([
    efn.EfficientNetB0(include_top=False, input_shape=(224,224,3), pooling='avg'),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(train_data.num_classes, activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model.summary()

# Train model
history = cnn_model.fit(train_data, validation_data=val_data, epochs=10)

# Save the trained model
cnn_model.save("plant_disease_model.h5")
print("🌿 Plant Disease Detection Model saved successfully!")

# ===============================
# 4️⃣ Part B: Crop Yield Prediction (Regression)
# ===============================

# Simulate agricultural dataset
# Columns: temperature, rainfall, soil_ph, fertilizer_type, yield
data = {
    "temperature": np.random.uniform(20, 40, 200),
    "rainfall": np.random.uniform(50, 300, 200),
    "soil_ph": np.random.uniform(5.0, 8.0, 200),
    "fertilizer_type": np.random.choice([1, 2, 3], 200),  # encoded fertilizers
    "yield": np.random.uniform(1000, 5000, 200)
}
df = pd.DataFrame(data)

# Split data
X = df.drop("yield", axis=1)
y = df["yield"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build regression model (simple neural network)
reg_model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

reg_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
reg_model.summary()

# Train regression model
reg_model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=50, verbose=1)

# Evaluate model
y_pred = reg_model.predict(X_test_scaled).flatten()
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"🌾 Yield Prediction MAE: {mae:.2f}, R²: {r2:.2f}")

# ===============================
# 5️⃣ Save Predictions to MongoDB
# ===============================
for i in range(len(y_pred)):
    record = {
        "temperature": float(X_test.iloc[i, 0]),
        "rainfall": float(X_test.iloc[i, 1]),
        "soil_ph": float(X_test.iloc[i, 2]),
        "fertilizer_type": int(X_test.iloc[i, 3]),
        "predicted_yield": float(y_pred[i])
    }
    collection.insert_one(record)

print("📦 Predictions successfully stored in MongoDB!")

# ===============================
# 6️⃣ Visualization
# ===============================
plt.figure(figsize=(6,4))
sns.scatterplot(x=y_test, y=y_pred)
plt.xlabel("Actual Yield")
plt.ylabel("Predicted Yield")
plt.title("Actual vs Predicted Crop Yield")
plt.show()

print("✅ Model training and prediction completed successfully!")
